In [1]:
from keras.models import load_model
import numpy as np
import pandas as pd


In [2]:
from keras.models import load_model
model = load_model('modelv1.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 6, 64)             256       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 3, 64)            0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 192)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                12352     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,673
Trainable params: 12,673
Non-trainable params: 0
__________________________________________________

In [3]:
def get_value_conditions(conditions):
  if conditions == "Clear":
    return [1, 0, 0, 0, 0]
  elif conditions == "Overcast":
    return [0, 1, 0, 0, 0]
  elif conditions == "Partially cloudy":
    return [0, 0, 1, 0, 0]
  elif conditions == "Rain, Overcast":
    return [0, 0, 0, 1, 0]
  elif conditions == "Rain, Partially cloudy":
    return [0, 0, 0, 0, 1]

In [4]:
# Data baru yang ingin diprediksi
condition = get_value_conditions("Rain, Partially cloudy")
temp = np.array([25.9])
feels_like  = np.array([26.9])
libur = np.array([1])


df = pd.DataFrame({
    'temp': temp,
    'feelslike': feels_like,
    'libur': libur,
    'conditions_Clear': condition[0],
    'conditions_Overcast': condition[1],
    'conditions_Partially cloudy': condition[2],
    'conditions_Rain, Overcast': condition[3],
    'conditions_Rain, Partially cloudy': condition[4],
})
df

,temp,feelslike,libur,conditions_Clear,conditions_Overcast,conditions_Partially cloudy,"conditions_Rain, Overcast","conditions_Rain, Partially cloudy"
0,25.9,26.9,1,0,0,0,0,1


In [5]:
# # Memperoleh prediksi daya listrik dari model
prediction = model.predict(df)

# # Menampilkan hasil prediksi
print(prediction)


1/1 [==============================] - 0s 458ms/step
[[1.6567924]]


## get holidays data

In [8]:
from datetime import date, timedelta, datetime
import holidays


class Get_Data():
    def __init__(self):
        self.get_holiday_instance = Get_Holiday()
        # self.get_weather_instance = Get_Weather()

    def get_holiday_range(self, count: int) -> list[int]:
        "mendapatkan range list dengan nilai 0 dan 1 tanggal dari hari ini sampai (count) hari"
        return self.get_holiday_instance.get_holiday_range(count)

    # def get_weather(self):
        # self.get_weather_instance.get_weather()

class date_range():
    def __init__(self) -> None:
        self.date = date.today()

    def get_date_range(self, count: int) -> list[str]:
        """mendapatkan range tanggal dari hari ini sampai (count) hari"""
        return [str(self.date + timedelta(days=i))
                for i in range(count)]

class Get_Holiday(date_range):
    def __init__(self) -> None:
        self.date = date.today()
        self.year = date.today().year

    def get_holiday(self):
        # Buat objek Holidays dengan kode "ID" yang merupakan kode negara untuk Indonesia
        holiday = holidays.Indonesia(years=self.year)
        id_holidays = holidays.CountryHoliday('ID')

        # Mendapatkan daftar hari libur akhir pekan
        weekend_holidays = []

        # Iterasi melalui setiap tanggal dalam rentang tahun
        start_date = date(self.year, 1, 1)
        end_date = date(self.year, 12, 31)

        while start_date <= end_date:
            # Periksa apakah tanggal adalah hari Minggu
            if start_date.weekday() == 6:  # 6 mewakili hari Minggu
                weekend_holidays.append(start_date)

            start_date += timedelta(days=1)

        # Tambahkan hari libur akhir pekan ke dalam objek holiday
        for holiday in weekend_holidays:
            id_holidays[holiday] = "Weekend Holiday"
        return id_holidays

    def get_holiday_range(self, count: int) -> list[int]:
        holidays = self.get_holiday()
        # jika libur berikan output 1 dan jika tidak output 0
        output = []
        for date in self.get_date_range(count):
            if date in holidays:
                output.append(1)
            else:
                output.append(0)

        return output

class Get_Weather(date_range):
    def __init__(self) -> None:
        pass

    def get_weather(self):
        pass


data = Get_Data()

print(data.get_holiday_range(10))


[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [6]:
from datetime import date


class Get_Data:
    def __init__(self):
        self.date = date.today()
        self.year = date.today().year
        self.get_holiday_instance = Get_Holiday()
        self.get_weather_instance = Get_Weather()

    def get_holiday(self):
        self.get_holiday_instance.get_holiday()

    def get_weather(self):
        self.get_weather_instance.get_weather()


class Get_Holiday():
    def __init__(self) -> None:
        super().__init__()

    def get_holiday(self):
        print("Mengakses metode get_holiday dari Get_Holiday")


class Get_Weather:
    def get_weather(self):
        print("Mengakses metode get_weather dari Get_Weather")


data = Get_Data()
data.get_holiday()  # Mengakses metode get_holiday dari Get_Holiday
data.get_weather()  # Mengakses metode get_weather dari Get_Weather


Mengakses metode get_holiday dari Get_Holiday
Mengakses metode get_weather dari Get_Weather


In [ ]:
import csv
import requests


def get_weather_forecast(city):
    # Ganti dengan API key yang Anda dapatkan dari OpenWeather
    api_key = "11a5ba73255cfa55831c342c4a9cceee"
    base_url = "http://api.openweathermap.org/data/2.5/forecast"

    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",
        "cnt": 1  # Mengambil prediksi cuaca satu minggu ke depan (7 hari)
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    print(data)

    if data["cod"] == "200":
        print("Prediksi cuaca untuk", data["city"]["name"])

        for forecast in data["list"]:
            date = forecast["dt_txt"]
            temperature = forecast["main"]["temp"]
            weather_desc = forecast["weather"][0]["description"]

            print(
                f"Tanggal: {date}, Temperatur: {temperature}°C, Deskripsi Cuaca: {weather_desc}")
    else:
        print("Gagal memperoleh data cuaca.")


# Memanggil fungsi get_weather_forecast dengan kota yang diinginkan
# get_weather_forecast("Banda Aceh")


def get_weather_data(city):
    # Ganti dengan API key yang Anda dapatkan dari OpenWeather
    api_key = "11a5ba73255cfa55831c342c4a9cceee"

    base_url = "http://api.openweathermap.org/data/2.5/weather"

    start_date = "2023-01-01"
    end_date = "2023-05-31"

    date = start_date
    with open("weather_data.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Temperature (°C)", "Weather Description"])

        while date <= end_date:
            params = {
                "q": city,
                "appid": api_key,
                "units": "metric",
                "dt": date
            }

            response = requests.get(base_url, params=params)
            data = response.json()

            if data["cod"] == 200:
                temperature = data["main"]["temp"]
                weather_desc = data["weather"][0]["description"]
                writer.writerow([date, temperature, weather_desc])

            date = increment_date(date)
            print(date)

    print("Data cuaca berhasil disimpan dalam weather_data.csv")


def increment_date(date):
    year, month, day = map(int, date.split("-"))
    next_day = day + 1

    # Handle end of month
    if next_day > 31 or (month == 2 and next_day > 28):
        next_day = 1
        month += 1

    # Handle end of year
    if month > 12:
        month = 1
        year += 1

    return f"{year:04d}-{month:02d}-{next_day:02d}"


# Memanggil fungsi get_weather_data dengan kota yang diinginkan
get_weather_data("Banda Aceh")
